# <u><center>CREACIÓN DE LA BASE DE DATOS (DDBB)</center></u>

En este notebook se realiza todo el procesamiento de las mutaciones recopiladas (eliminación de duplicados, homogeneización del formato...) así como la propia creación de la base de datos, junto con todos sus descriptores.

<br>
<br>

____

**CONTENIDOS**
* [Creación de la base de datos final](#Ap1) <br>
    * [1. Caracterización de las mutaciones mediante descriptores](#1.1)
    * [2. Eliminación de duplicados, unificación del formato y corrección de errores](#1.2)
    * [3. Preprocesado final para los modelos de ML](#1.3)
      
___  
   

In [1]:
# Packages needed
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

<br>
<br>

##  Creación de la base de datos final <a id= "Ap1"> </a>


En primer lugar, se carga el **archivo CSV con las variantes de KCNQ2** recopiladas, ya sea por fuentes bibliográficas o mediante búsqueda en bases de datos. 

In [2]:
# Define a df
df = pd.read_csv('FINAL_DDBB.csv', header = "infer")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   PrimaryDisease  353 non-null    object
 1   Mutationppt     353 non-null    object
dtypes: object(2)
memory usage: 5.6+ KB


El archivo de origen presenta solo 2 columnas y 353 filas. La columna `df[PrimaryDisease]` hace referencia a la **etiqueta** de la mutación, es decir, a si está registrada como patogénica o benigna. La segunda columna `df[Mutationppt]` recoge la **mutación**, con una nomenclatura específica.

In [4]:
df.head()

,PrimaryDisease,Mutationppt
0,pathogenic_variant,M1T
1,pathogenic_variant,M1V
2,pathogenic_variant,L107F
3,pathogenic_variant,T114A
4,pathogenic_variant,T114I


Para saber el número de mutaciones benignas y patológicas presentes en el fichero:

In [5]:
df["PrimaryDisease"].value_counts()

pathogenic_variant    314
benign_variant         39
Name: PrimaryDisease, dtype: int64

<br>
<br>
<br> 

### 1. Caracterización de las mutaciones mediante descriptores  <a id="1.1"></a>

En este apartado, se diseñan los distintos descriptores de secuencia y estructurales con el fin de caracterizar las mutaciones de KCNQ2 recopiladas.

<br>

<u> **initial_aa, final_aa, position_aa** </u>

Una vez se ha realizado el filtrado de las mutaciones y la corrección de su formato (realizado en pasos previos), se crean tres nuevas columnas para caracterizar las mutaciones puntuales **a nivel de la secuencia proteica**:
* `df["initial_aa"]` almacenará el aminoácido inicial antes de la mutación.
* `df["final_aa"]` almacenará el aminoácido final tras la mutación.
* `df["position_aa"]` almacenará la posición de la secuencia proteica donde ha ocurrido la mutación.

In [6]:
df['initial_aa'] = df['Mutationppt'].apply(lambda x: x[0])
df['final_aa'] = df['Mutationppt'].apply(lambda x: x[-1] if x[-3:] != "del" else "-")
df['position_aa'] = df["Mutationppt"].apply(lambda x: int(x[1:-1]) if x[-3:] != "del" else int(x[1:-3]))

Se comprueban los valores únicos de las columnas creadas para ver que no ha habido errores:

In [7]:
df["final_aa"].unique()

array(['T', 'V', 'F', 'A', 'I', 'G', 'L', 'K', 'Q', 'D', 'R', 'E', 'Y',
       'S', 'P', 'C', 'H', 'W', 'N', 'M'], dtype=object)

In [8]:
df["initial_aa"].unique()

array(['M', 'L', 'T', 'E', 'S', 'R', 'Y', 'G', 'V', 'N', 'A', 'I', 'D',
       'H', 'W', 'P', 'F', 'Q', 'K', 'C'], dtype=object)

Ahora, el DataFrame tiene la siguiente forma:

In [9]:
df.head()

,PrimaryDisease,Mutationppt,initial_aa,final_aa,position_aa
0,pathogenic_variant,M1T,M,T,1
1,pathogenic_variant,M1V,M,V,1
2,pathogenic_variant,L107F,L,F,107
3,pathogenic_variant,T114A,T,A,114
4,pathogenic_variant,T114I,T,I,114


<br>
<br>

<u> **Localización topológica y dominios funcionales donde ocurren las mutaciones** </u>

A partir de la columna creada de `df["position_aa"]`, que registra la posición donde ocurre la mutación, se genera el descriptor `df["topological_domain"]` con los dominios topológicos a los que corresponderían.

In [10]:
df.loc[(df["position_aa"] >= 1) & (df["position_aa"] <= 91), "topological_domain"] = "Cytoplasmic"
df.loc[(df["position_aa"] >= 92) & (df["position_aa"] <= 112), "topological_domain"] = "S1"
df.loc[(df["position_aa"] >= 113) & (df["position_aa"] <= 122), "topological_domain"] = "Extracelullar"
df.loc[(df["position_aa"] >= 123) & (df["position_aa"] <= 143), "topological_domain"] = "S2"
df.loc[(df["position_aa"] >= 144) & (df["position_aa"] <= 166), "topological_domain"] = "Cytoplasmic"
df.loc[(df["position_aa"] >= 167) & (df["position_aa"] <= 187), "topological_domain"] = "S3"
df.loc[(df["position_aa"] >= 188) & (df["position_aa"] <= 195), "topological_domain"] = "Extracelullar"
df.loc[(df["position_aa"] >= 196) & (df["position_aa"] <= 218), "topological_domain"] = "S4"
df.loc[(df["position_aa"] >= 219) & (df["position_aa"] <= 231), "topological_domain"] = "Cytoplasmic"
df.loc[(df["position_aa"] >= 232) & (df["position_aa"] <= 252), "topological_domain"] = "S5"
df.loc[(df["position_aa"] >= 253) & (df["position_aa"] <= 264), "topological_domain"] = "Extracelullar"
df.loc[(df["position_aa"] >= 265) & (df["position_aa"] <= 285), "topological_domain"] = "Pore"
df.loc[(df["position_aa"] >= 286) & (df["position_aa"] <= 291), "topological_domain"] = "Extracelullar"
df.loc[(df["position_aa"] >= 292) & (df["position_aa"] <= 312), "topological_domain"] = "S6"
df.loc[(df["position_aa"] >= 313) & (df["position_aa"] <= 331), "topological_domain"] = "Cytoplasmic"
df.loc[(df["position_aa"] >= 332) & (df["position_aa"] <= 350), "topological_domain"] = "hA"
df.loc[(df["position_aa"] >= 351) & (df["position_aa"] <= 356), "topological_domain"] = "Cytoplasmic"
df.loc[(df["position_aa"] >= 357) & (df["position_aa"] <= 366), "topological_domain"] = "hTW"
df.loc[(df["position_aa"] >= 367) & (df["position_aa"] <= 534), "topological_domain"] = "Cytoplasmic"
df.loc[(df["position_aa"] >= 535) & (df["position_aa"] <= 559), "topological_domain"] = "hB"
df.loc[(df["position_aa"] >= 560) & (df["position_aa"] <= 562), "topological_domain"] = "Cytoplasmic"
df.loc[(df["position_aa"] >= 563) & (df["position_aa"] <= 594), "topological_domain"] = "hC"
df.loc[(df["position_aa"] >= 595) & (df["position_aa"] <= 621), "topological_domain"] = "Cytoplasmic"
df.loc[(df["position_aa"] >= 622) & (df["position_aa"] <= 647), "topological_domain"] = "hD"
df.loc[(df["position_aa"] >= 648) & (df["position_aa"] <= 872), "topological_domain"] = "Cytoplasmic"

Se comprueban los valores únicos de la columna creada para ver que no ha habido errores:

In [11]:
df["topological_domain"].unique()

array(['Cytoplasmic', 'S1', 'Extracelullar', 'S2', 'S3', 'S4', 'S5',
       'Pore', 'S6', 'hA', 'hTW', 'hB', 'hC', 'hD'], dtype=object)

Siguiendo el mismo procedimiento que el descriptor anterior, se añade la información del dominio funcional correspondiente (`df["functional_domain"]`).

In [12]:
df.loc[(df["position_aa"] >= 1) & (df["position_aa"] <= 91), "functional_domain"] = "unknown_function"
df.loc[(df["position_aa"] >= 92) & (df["position_aa"] <= 218), "functional_domain"] = "voltage_domain"
df.loc[(df["position_aa"] >= 219) & (df["position_aa"] <= 231), "functional_domain"] = "unknown_function"
df.loc[(df["position_aa"] >= 232) & (df["position_aa"] <= 276), "functional_domain"] = "pore_domain"
df.loc[(df["position_aa"] >= 277) & (df["position_aa"] <= 282), "functional_domain"] = "selectivity_filter"
df.loc[(df["position_aa"] >= 283) & (df["position_aa"] <= 312), "functional_domain"] = "pore_domain"
df.loc[(df["position_aa"] >= 313) & (df["position_aa"] <= 331), "functional_domain"] = "unknown_function"
df.loc[(df["position_aa"] >= 332) & (df["position_aa"] <= 559), "functional_domain"] = "CaM_interaction"
df.loc[(df["position_aa"] >= 560) & (df["position_aa"] <= 562), "functional_domain"] = "unknown_function"
df.loc[(df["position_aa"] >= 563) & (df["position_aa"] <= 647), "functional_domain"] = "SID_domain"
df.loc[(df["position_aa"] >= 648) & (df["position_aa"] <= 872), "functional_domain"] = "unknown_function"

Se comprueban los valores únicos de la columna creada para ver que no ha habido errores:

In [13]:
df["functional_domain"].unique()

array(['unknown_function', 'voltage_domain', 'pore_domain',
       'selectivity_filter', 'CaM_interaction', 'SID_domain'],
      dtype=object)

<br>
<br>
<br>

<u> **char_initial_aa, char_final_aa** </u>

Ahora, se van a crear otras dos columnas para hacer referencia al cambio de carga antes y después de la mutación. Primero, se va a crear `df["char_initial_aa"]` y `df["char_final_aa"]` a partir de `df["initial_aa"]` y `df["final_aa"]`, respectivamente. Los datos se han obtenido de Lodish, University Harvey, Berk, University Arnold, Kaiser, University Chris A, Krieger, University Monty, Bretscher, University Anthony, Ploegh, University Hidde, Scott, M. P., & Amon, A. (2012). Molecular Cell Biology (7th ed.). W. H. Freeman.

In [14]:
df['char_initial_aa'] = df['initial_aa']
df['char_final_aa'] = df['final_aa']

charge_map = {
    'S':'neutral',
    'T':'neutral',
    'Q':'neutral',
    'N':'neutral',
    'Y':'neutral', 
    'C':'neutral',
    'G':'neutral',
    'A':'neutral',
    'V':'neutral',
    'L':'neutral',
    'I':'neutral',
    'M':'neutral',
    'P':'neutral',
    'F':'neutral',
    'W':'neutral',
    'D':'negative_acidic',
    'E':'negative_acidic', 
    'K':'positive_basic',
    'R':'positive_basic',
    'H':'positive_basic'
}

Se comprueban los valores únicos de la columna creada para ver que no ha habido errores:

In [15]:
# char_initial_aa
df["char_initial_aa"].replace(charge_map, inplace=True)
print(df["char_initial_aa"].unique())

# char_final_aa
df["char_final_aa"].replace(charge_map, inplace=True)
print(df["char_final_aa"].unique())

['neutral' 'negative_acidic' 'positive_basic']
['neutral' 'positive_basic' 'negative_acidic']


<br>
<br>
<br>

<u> **pol_initial_aa, pol_final_aa** </u>

Se van a crear otras dos columnas para hacer referencia al cambio de polaridad antes y después de la mutación a nivel de aminoácido. Primero, se va a crear `df["pol_initial_aa"]` y `df["pol_final_aa"]` a partir de `df["initial_aa"]` y `df["final_aa"]`, respectivamente. Después, se procederá a la sustitución de sus valores mediante un mapeo.

In [16]:
df['pol_initial_aa'] = df['initial_aa']
df['pol_final_aa'] = df['final_aa']


pol_map = {
    'S':'polar',
    'T':'polar',
    'Q':'polar',
    'N':'polar',
    'Y':'non_polar', 
    'C':'polar',
    'G':'polar',
    'A':'non_polar',
    'V':'non_polar',
    'L':'non_polar',
    'I':'non_polar',
    'M':'non_polar',
    'P':'non_polar',
    'F':'non_polar', 
    'W':'non_polar', 
    'D':'polar',
    'E':'polar', 
    'K':'polar',
    'R':'polar',
    'H':'polar'   
}

Se comprueban los valores únicos de la columna creada para ver que no ha habido errores:

In [17]:
# pol_initial_aa
df["pol_initial_aa"].replace(pol_map, inplace=True)
print(df["pol_initial_aa"].unique())

# pol_final_aa
df["pol_final_aa"].replace(pol_map, inplace=True)
print(df["pol_final_aa"].unique())

['non_polar' 'polar']
['polar' 'non_polar']


<br>
<br>
<br>

<u> **aro_initial_aa, aro_final_aa** </u>

Se van a crear otras dos columnas para hacer referencia al cambio de aromaticidad antes y después de la mutación a nivel aminoácido. Primero, se va a crear `df["aro_initial_aa"]` y `df["aro_final_aa"]` a partir de `df["initial_aa"]` y `df["final_aa"]`, respectivamente. Después, se procederá a la sustitución de sus valores mediante un mapeo.

In [18]:
df['aro_initial_aa'] = df['initial_aa']
df['aro_final_aa'] = df['final_aa']


aro_map = {
    'S':'non_aromatic',
    'T':'non_aromatic',
    'Q':'non_aromatic',
    'N':'non_aromatic',
    'Y':'aromatic', 
    'C':'non_aromatic',
    'G':'non_aromatic',
    'A':'non_aromatic',
    'V':'non_aromatic',
    'L':'non_aromatic',
    'I':'non_aromatic',
    'M':'non_aromatic',
    'P':'non_aromatic',
    'F':'aromatic', 
    'W':'aromatic',
    'D':'non_aromatic',
    'E':'non_aromatic', 
    'K':'non_aromatic',
    'R':'non_aromatic',
    'H':'non_aromatic'  
}

Se comprueban los valores únicos de la columna creada para ver que no ha habido errores:

In [19]:
# aro_initial_aa
df["aro_initial_aa"].replace(aro_map, inplace=True)
print(df["aro_initial_aa"].unique())

# aro_final_aa
df["aro_final_aa"].replace(aro_map, inplace=True)
print(df["aro_final_aa"].unique())

['non_aromatic' 'aromatic']
['non_aromatic' 'aromatic']


<br>
<br>
<br>

<u> **Información molecular cuantitativa de los aminoácidos** </u>


A continuación, se van a crear columnas adicionales para introducir datos moleculares de los aminoácidos iniciales y los que han sido sustituidos. 

* `df["mw_initial_aa"]`. Recoge los pesos moleculares (*molecular weights*) en Dalton de los aminoácidos iniciales. 
* `df["mw_final_aa"]`. Recoge los pesos moleculares (*molecular weights*) en Dalton de los aminoácidos finales. 

Con ese fin, se emplea un diccionario que luego se mappeará. Se realiza el mismo procedimiento que el realizado con anterioridad. Se ha empleado como tabla de valores la obtenida de Lide, D. R. (2007). CRC Handbook of Chemistry and Physics, 88th Edition. Taylor & Francis. [Página 1180](https://fap.if.usp.br/~jhsevero/Fisica_Experimental_C_Semestral_2020/crc.pdf).

In [20]:
df['mw_initial_aa'] = df['initial_aa']
df['mw_final_aa'] = df['final_aa']


mw_map = {
    'S': 105.09,
    'T': 119.12,
    'Q': 146.15,
    'N': 132.12,
    'Y': 181.19, 
    'C': 121.16,
    'G': 75.07,
    'A': 89.09,
    'V': 117.15,
    'L': 131.17,
    'I': 131.17,
    'M': 149.21,
    'P': 115.13,
    'F': 165.19, 
    'W': 204.23,
    'D': 133.10 ,
    'E': 147.13, 
    'K': 146.19,
    'R': 174.20,
    'H': 155.16,
    
}

Se comprueban los valores únicos de la columna creada para ver que no ha habido errores:

In [21]:
# mw_initial_aa
df["mw_initial_aa"].replace(mw_map, inplace=True)
print(df["mw_initial_aa"].unique())

# mw_final_aa
df["mw_final_aa"].replace(mw_map, inplace=True)
print(df["mw_final_aa"].unique())

[149.21 131.17 119.12 147.13 105.09 174.2  181.19  75.07 117.15 132.12
  89.09 133.1  155.16 204.23 115.13 165.19 146.15 146.19 121.16]
[119.12 117.15 165.19  89.09 131.17  75.07 146.19 146.15 133.1  174.2
 147.13 181.19 105.09 115.13 121.16 155.16 204.23 132.12 149.21]


<br>

Se van a crear una serie de descriptores nuevos basándose en Bogardt, R. A., Jones, B. N., Dwulet, F. E., Garner, W. H., Lehman, L. D., & Gurd, F. R. N. (1980). Evolution of the amino acid substitution in the mammalian myoglobin gene. Journal of Molecular Evolution, 15(3), 197–218. https://doi.org/10.1007/bf01732948. Todas las características aparecen en % estandarizados.

* `df["v_e_initial_aa"]`. Recoge los volúmenes estandarizados (en %) de los aminoácidos iniciales.
* `df["v_e_final_aa"]`. Recoge los volúmenes estandarizados (en %) de los aminoácidos finales.
Con ese fin, se emplea un diccionario que luego se mappeará. Se realiza el mismo procedimiento que el realizado con anterioridad:

In [22]:
df['v_e_initial_aa'] = df['initial_aa']
df['v_e_final_aa'] = df['final_aa']


v_map = {
    'K': 68.0,
    'H': 49.2,
    'R': 70.8,
    'D': 31.3,
    'E': 47.2,
    'N': 35.4,
    'Q': 51.3,
    'S': 18.1,
    'T': 34.0,
    'C': 28.0,
    'G': 0.0,
    'A': 15.9,
    'P': 41.0,
    'V': 47.7,
    'M': 62.8,
    'I': 63.6,
    'L': 63.6,
    'Y': 78.5,
    'F': 77.2,
    'W': 100.0
    
}

Se comprueban los valores únicos de la columna creada para ver que no ha habido errores:

In [23]:
# v_e_initial_aa
df["v_e_initial_aa"].replace(v_map, inplace=True)
print(df["v_e_initial_aa"].unique())

# v_e_final_aa
df["v_e_final_aa"].replace(v_map, inplace=True)
print(df["v_e_final_aa"].unique())

[ 62.8  63.6  34.   47.2  18.1  70.8  78.5   0.   47.7  35.4  15.9  31.3
  49.2 100.   41.   77.2  51.3  68.   28. ]
[ 34.   47.7  77.2  15.9  63.6   0.   68.   51.3  31.3  70.8  47.2  78.5
  18.1  41.   28.   49.2 100.   35.4  62.8]


<br>

* `df["pol_e_initial_aa"]`. Recoge la polaridad estandarizada (en %) de los aminoácidos iniciales. Dato cuantitativo.
* `df["pol_e_final_aa"]`. Recoge la polaridad estandarizada (en %) de los aminoácidos finales. Dato cuantitativo.


Con ese fin, se emplea un diccionario que luego se mappeará. Se realiza el mismo procedimiento que el realizado con anterioridad:

In [24]:
df['pol_e_initial_aa'] = df['initial_aa']
df['pol_e_final_aa'] = df['final_aa']


p_map = {
    'K': 64.2,
    'H': 43.2,
    'R': 51.9,
    'D': 100.0,
    'E': 93.8,
    'N': 63.0,
    'Q': 45.7,
    'S': 32.1,
    'T': 21.0,
    'C': 7.4,
    'G': 37.0,
    'A': 25.9,
    'P': 21.0,
    'V': 8.6,
    'M': 4.9,
    'I': 0.0,
    'L': 0.0,
    'Y': 9.9,
    'F': 1.2,
    'W': 4.9
    
}

Se comprueban los valores únicos de la columna creada para ver que no ha habido errores:

In [25]:
# pol_e_initial_aa
df["pol_e_initial_aa"].replace(p_map, inplace=True)
print(df["pol_e_initial_aa"].unique())

# pol_e_final_aa
df["pol_e_final_aa"].replace(p_map, inplace=True)
print(df["pol_e_final_aa"].unique())

[  4.9   0.   21.   93.8  32.1  51.9   9.9  37.    8.6  63.   25.9 100.
  43.2   1.2  45.7  64.2   7.4]
[ 21.    8.6   1.2  25.9   0.   37.   64.2  45.7 100.   51.9  93.8   9.9
  32.1   7.4  43.2   4.9  63. ]


<br>

* `df["ip_e_initial_aa"]`. Recoge el punto isoeléctrico estandarizado (en %) de los aminoácidos iniciales. 
* `df["ip_e_final_aa"]`. Recoge el punto isoeléctrico estandarizado  (en %) de los aminoácidos finales. 


Con ese fin, se emplea un diccionario que luego se mappeará. Se realiza el mismo procedimiento que el realizado con anterioridad:

In [26]:
df['ip_e_initial_aa'] = df['initial_aa']
df['ip_e_final_aa'] = df['final_aa']


ip_e_map = {
    'K': 86.9 ,
    'H': 59.2,
    'R': 100.0,
    'D': 0.0,
    'E': 3.2,
    'N': 31.3,
    'Q': 34.4,
    'S': 34.8,
    'T': 45.7,
    'C': 26.3,
    'G': 38.5,
    'A': 39.2,
    'P': 40.2,
    'V': 38.5,
    'M': 35.7,
    'I': 39.2,
    'L': 38.6,
    'Y': 34.4,
    'F': 38.6,
    'W': 37.7
    
}

Se comprueban los valores únicos de la columna creada para ver que no ha habido errores:

In [27]:
# ip_e_initial_aa
df["ip_e_initial_aa"].replace(ip_e_map, inplace=True)
print(df["ip_e_initial_aa"].unique())

# ip_e_final_aa
df["ip_e_final_aa"].replace(ip_e_map, inplace=True)
print(df["ip_e_final_aa"].unique())

[ 35.7  38.6  45.7   3.2  34.8 100.   34.4  38.5  31.3  39.2   0.   59.2
  37.7  40.2  86.9  26.3]
[ 45.7  38.5  38.6  39.2  86.9  34.4   0.  100.    3.2  34.8  40.2  26.3
  59.2  37.7  31.3  35.7]


<br>

* `df["hf_e_initial_aa"]`. Recoge la hidrofobicidad estandarizada (en %) de los aminoácidos iniciales. 
* `df["hf_e_final_aa"]`. Recoge la hidrofobicidad estandarizada (en %) de los aminoácidos finales. 


Con ese fin, se emplea un diccionario que luego se mappeará. Se realiza el mismo procedimiento que el realizado con anterioridad:

In [28]:
df['hf_e_initial_aa'] = df['initial_aa']
df['hf_e_final_aa'] = df['final_aa']


hf_e_map = {
    'K': 43.5,
    'H': 23.1,
    'R': 22.6,
    'D': 17.5,
    'E': 17.8,
    'N': 2.4,
    'Q': 0.0,
    'S': 1.9,
    'T': 1.9,
    'C': 40.3,
    'G': 2.7,
    'A': 23.1,
    'P': 73.5,
    'V': 49.6,
    'M': 44.3,
    'I': 83.6,
    'L': 57.6,
    'Y': 70.8,
    'F': 76.1,
    'W': 100.0 
    
}

Se comprueban los valores únicos de la columna creada para ver que no ha habido errores:

In [29]:
# hf_e_initial_aa
df["hf_e_initial_aa"].replace(hf_e_map, inplace=True)
print(df["hf_e_initial_aa"].unique())

# hf_e_final_aa
df["hf_e_final_aa"].replace(hf_e_map, inplace=True)
print(df["hf_e_final_aa"].unique())

[ 44.3  57.6   1.9  17.8  22.6  70.8   2.7  49.6   2.4  23.1  83.6  17.5
 100.   73.5  76.1   0.   43.5  40.3]
[  1.9  49.6  76.1  23.1  83.6   2.7  57.6  43.5   0.   17.5  22.6  17.8
  70.8  73.5  40.3 100.    2.4  44.3]


<br>

* `df["msa_e_initial_aa"]`. Recoge la accesibilidad media del solvente estandarizada (en %) de los aminoácidos iniciales. 
* `df["msa_e_final_aa"]`. Recoge la accesibilidad media del solvente estandarizada (en %) de los aminoácidos finales. 

Con ese fin, se emplea un diccionario que luego se mappeará. Se realiza el mismo procedimiento que el realizado con anterioridad:

In [30]:
df['msa_e_initial_aa'] = df['initial_aa']
df['msa_e_final_aa'] = df['final_aa']


msa_map = {
    'K': 54.3,
    'H': 28.1,
    'R': 50.1,
    'D': 45.0,
    'E': 48.6,
    'N': 46.1,
    'Q': 43.6,
    'S': 40.5,
    'T': 35.3,
    'C': 7.4,
    'G': 54.0,
    'A': 37.4,
    'P': 66.2,
    'V': 19.6,
    'M': 3.9,
    'I': 7.5,
    'L': 10.1,
    'Y': 30.1,
    'F': 5.5,
    'W': 13.8 
    
}

Se comprueban los valores únicos de la columna creada para ver que no ha habido errores:

In [31]:
# msa_e_initial_aa
df["msa_e_initial_aa"].replace(msa_map, inplace=True)
print(df["msa_e_initial_aa"].unique())

# msa_e_final_aa
df["msa_e_final_aa"].replace(msa_map, inplace=True)
print(df["msa_e_final_aa"].unique())

[ 3.9 10.1 35.3 48.6 40.5 50.1 30.1 54.  19.6 46.1 37.4  7.5 45.  28.1
 13.8 66.2  5.5 43.6 54.3  7.4]
[35.3 19.6  5.5 37.4  7.5 54.  10.1 54.3 43.6 45.  50.1 48.6 30.1 40.5
 66.2  7.4 28.1 13.8 46.1  3.9]


<br>

**NOTA**. No existe ningún error de los que se observaban en anteriores notebooks. Se han corregido todos los errores de formato. Si al ampliar la tabla se respeta el formato del archivo introducido no habrá problemas en la ampliación de la tabla a lo ancho (ampliación de descriptores).

<br>

Se van a añadir dos columnas adicionales que hacen referencia a los valores de hidrofobicidad de los aminoácidos sin estandarizar:

* `df["hf_initial_aa"]`. Recoge la hidrofobicidad de los aminoácidos iniciales. 
* `df["hf_final_aa"]`. Recoge la hidrofobicidad de los aminoácidos finales. 

Se toma como referencia el paper de Kyte J, Doolittle RF. A simple method for displaying the hydropathic character of a protein. J. Mol. Biol. 1982;157:105–132.

In [32]:
df['hf_initial_aa'] = df['initial_aa']
df['hf_final_aa'] = df['final_aa']


hf_map = {
    'K': -3.9,
    'H': -3.2,
    'R': -4.5,
    'D': -3.5,
    'E': -3.5,
    'N': -3.5,
    'Q': -3.5,
    'S': -0.8,
    'T': -0.7,
    'C': 2.5,
    'G': -0.4,
    'A': 1.8,
    'P': -1.6,
    'V': 4.2,
    'M': 1.9,
    'I': 4.5,
    'L': 3.8,
    'Y': -1.3,
    'F': 2.8,
    'W': -0.9
    
}

Se comprueban los valores únicos de la columna creada para ver que no ha habido errores:

In [33]:
# hf_initial_aa
df["hf_initial_aa"].replace(hf_map, inplace=True)
print(df["hf_initial_aa"].unique())

# hf_final_aa
df["hf_final_aa"].replace(hf_map, inplace=True)
print(df["hf_final_aa"].unique())

[ 1.9  3.8 -0.7 -3.5 -0.8 -4.5 -1.3 -0.4  4.2  1.8  4.5 -3.2 -0.9 -1.6
  2.8 -3.9  2.5]
[-0.7  4.2  2.8  1.8  4.5 -0.4  3.8 -3.9 -3.5 -4.5 -1.3 -0.8 -1.6  2.5
 -3.2 -0.9  1.9]


<br>

<u> **Columnas de cambio** </u>


Se van a crear descriptores adicionales según el paper de [Asier Larrea *et al.*, 2021](https://pubmed.ncbi.nlm.nih.gov/34869944/). Estas columnas adicionales de cambio recogerán la variación entre el valor del aminoácido incial y del aminoácido mutado. Concretamente, se crearán las siguientes columnas:

* `df["d_charge"]`: recoge el cambio de carga tras la mutación. Necesita codificación al ser datos cualitativos. Emplea como base las columnas `df["char_initial_aa"]` y `df["char_final_aa"]`.
* `df["d_pol"]`: recoge el cambio de polaridad tras la mutación. Necesita codificación al ser datos cualitativos. Emplea como base las columnas `df["pol_initial_aa"]` y `df["pol_final_aa"]`.
* `df["d_aro"]`: recoge el cambio de polaridad tras la mutación. Necesita codificación al ser datos cualitativos. Emplea como base las columnas `df["aro_initial_aa"]` y `df["aro_final_aa"]`.
* `df["d_size"]`: recoge el cambio de tamaño tras la mutación. NO necesita codificación al ser un dato cuantitativo. Emplea como base los descriptores `df["mw_initial_aa"]` y `df["mw_final_aa"]`.
* `df["d_hf"]`: recoge el cambio de hidrofobicidad tras la mutación. NO necesita codificación al ser un dato cuantitativo. Emplea como base los descriptores `df["hf_initial_aa"]` y `df["hf_final_aa"]`.
* `df["d_vol"]`: recoge el cambio de volumen tras la mutación. NO necesita codificación al ser un dato cuantitativo. Emplea como base los descriptores de volúmenes **estandarizados** `df["v_e_initial_aa"]` y `df["v_e_final_aa"]`.
* `df["d_msa"]`: recoge el cambio de accesibilidad media del solvente tras la mutación. NO necesita codificación al ser un dato cuantitativo. Emplea como base los descriptores de msa **estandarizados** `df["msa_e_initial_aa"]` y `df["msa_e_final_aa"]`.


Se empieza creando las columnas cuantitativas por no necesitar codificación previa. Ej.: el descriptor `df["d_size"]` se obtiene restando las columnas de peso molecular del aminoácido previo y posterior a la mutación, es decir, `df["mw_initial_aa"]` y `df["mw_final_aa"]`.

In [34]:
# d_size
df["d_size"] = df["mw_initial_aa"] - df["mw_final_aa"]

# d_hf
df["d_hf"] = df["hf_initial_aa"] - df["hf_final_aa"]

# d_vol
df["d_vol"] = df["v_e_initial_aa"] - df["v_e_final_aa"]

# d_hf
df["d_msa"] = df["msa_e_initial_aa"] - df["msa_e_final_aa"]

Se comprueban los valores únicos de la columna creada para ver que no ha habido errores:

In [35]:
df["d_msa"].unique()

array([-31.4, -15.7,   4.6,  -2.1,  27.8,  -5.4,  30.4,  -5.7,   6.5,
       -14.9,   3.9,   5.4,   9.5,  10.4,  34.4,   5.6, -25.7,  17.8,
       -56.1,  42.7,  22. , -12.1,  36.3,  -9. , -10.8, -15.5, -30.9,
       -28.8,  26.7, -34.4,   2.1,  40.2,  14.9, -16. ,  -3.6,   3.7,
       -21.5,  31.4,  46.6,   2. ,  22.7,  38.1,  13.5,   9. ,   9.6,
        -3.9, -16.6, -35. ,  -4.6, -40. , -36.3,  -9.5,  35. , -19. ,
        62.3, -22.6,   3.6,  19. ,  40. ,   8.2,   5.7,  25.7,  56.1,
        -1.1, -25.4,  -3.6,   3.6,  10.8,  -2.6,  -6.5, -16.1, -22.7,
        15.7,   6.4,  25.4,  -8.2,  38.6,  15.5, -33.5,  28.8,  -2. ,
         4.2, -30.4, -38.6, -11.2, -27.8,  -5.2,  -1.9,  -3.1, -46.6,
         5. ,  50.4,   7.6,  33.1,  -7.6, -10.7,   1.9, -18. , -38.1,
       -50.4, -42.7, -26.7,  30.9,  16.6, -17.8, -22. ,  24.6,  16.9,
         5.2,  12.1,  -4.2,  16.1, -40.2,  18. ,   6.2])

In [36]:
df["d_vol"].unique()

array([  28.8,   15.1,  -13.6,   18.1,  -29.6,   47.2,  -45.5,  -20.8,
         19.5,  -70.8,  -47.2,  -15.9,  -60.4,  -47.7,   17.3,  -22.9,
        -31.8,   22.6,   42.8,   21.6,   15.9,  -29.2,   31.3,   -1.4,
         -2.1,   -7. ,  -25.1,  -81.9,   47.7,  -18.1, -100. ,   43.1,
        -15.9,   36.4,   66. ,  -28.8,  -28. ,   29.3,   50.5,   -8.2,
        -31.3,   52.7,   70.8,   15.9,   59.1,   13.6,   -7.2,   29.2,
        -59.1,  -34. ,  -21.8,   10.3,   15.9,   34. ,    7.2,   32.6,
         20.8,   22.9,  -22.6,   -4.1,   16.4,   -0.8,    0.8,    1.4,
          0. ,  -19.5,   29.8,  -50.5,  -15.1,   72. ,  -16.4,  -32.6,
        -28.2,    2.1,   12.3,   25.1,  -29.3,   -2.8,   45.5,   28.2,
        -31.3,   29.6,   49.2,   -2.2,   28. ,   -4.1,    5.2,   15.4,
         -9.9,  -15.4,  -16.7,  -49.2,   14.4,    8.2,   13.8,   -5.2,
        -42.8,   81.9,    7. ,  -15.9,   31.8,  -21.6,    1.3,  -17.9,
          2.8,  -29.8,  100. ,  -13.8])

A continuación, se diseña la codificación de las columnas cualitativas. Esta codificación es necesaria para registrar los cambios. Ej.: para registrar el cambio de carga `df["d_charge"]`, si el aminoácido inicial era positivo (pos) y el final era negativo (neg), el cambio se registra como "pos_to_neg"; si el aminoácido inicial era positivo (pos) y el final también lo es, el NO cambio se registra como "pos_to_pos".

In [37]:
# d_charge:
df.loc[(df["char_initial_aa"] == "positive_basic") & (df["char_final_aa"] == "positive_basic"), "d_charge"] = "pos_to_pos"
df.loc[(df["char_initial_aa"] == "positive_basic") & (df["char_final_aa"] == "negative_acidic"), "d_charge"] = "pos_to_neg"
df.loc[(df["char_initial_aa"] == "positive_basic") & (df["char_final_aa"] == "neutral"), "d_charge"] = "pos_to_neu"
df.loc[(df["char_initial_aa"] == "negative_acidic") & (df["char_final_aa"] == "positive_basic"), "d_charge"] = "neg_to_pos"
df.loc[(df["char_initial_aa"] == "negative_acidic") & (df["char_final_aa"] == "negative_acidic"), "d_charge"] = "neg_to_neg"
df.loc[(df["char_initial_aa"] == "negative_acidic") & (df["char_final_aa"] == "neutral"), "d_charge"] = "neg_to_neu"
df.loc[(df["char_initial_aa"] == "neutral") & (df["char_final_aa"] == "positive_basic"), "d_charge"] = "neu_to_pos"
df.loc[(df["char_initial_aa"] == "neutral") & (df["char_final_aa"] == "negative_acidic"), "d_charge"] = "neu_to_neg"
df.loc[(df["char_initial_aa"] == "neutral") & (df["char_final_aa"] == "neutral"), "d_charge"] = "neu_to_neu"

# Check if everything was ok
df["d_charge"].unique()

array(['neu_to_neu', 'neg_to_neu', 'neg_to_pos', 'pos_to_neu',
       'neu_to_neg', 'neu_to_pos', 'pos_to_pos', 'neg_to_neg',
       'pos_to_neg'], dtype=object)

Con la misma idea, se genera los descriptores `df["d_pol"]` y  `df["d_aro"]`

In [38]:
# d_pol:
df.loc[(df["pol_initial_aa"] == "polar") & (df["pol_final_aa"] == "polar"), "d_pol"] = "p_to_p"
df.loc[(df["pol_initial_aa"] == "polar") & (df["pol_final_aa"] == "non_polar"), "d_pol"] = "p_to_np"
df.loc[(df["pol_initial_aa"] == "non_polar") & (df["pol_final_aa"] == "polar"), "d_pol"] = "np_to_p"
df.loc[(df["pol_initial_aa"] == "non_polar") & (df["pol_final_aa"] == "non_polar"), "d_pol"] = "np_to_np"

# Check if everything was ok
df["d_pol"].unique()

array(['np_to_p', 'np_to_np', 'p_to_np', 'p_to_p'], dtype=object)

In [39]:
# d_aro:
df.loc[(df["aro_initial_aa"] == "aromatic") & (df["aro_final_aa"] == "aromatic"), "d_aro"] = "a_to_a"
df.loc[(df["aro_initial_aa"] == "aromatic") & (df["aro_final_aa"] == "non_aromatic"), "d_aro"] = "a_to_na"
df.loc[(df["aro_initial_aa"] == "non_aromatic") & (df["aro_final_aa"] == "aromatic"), "d_aro"] = "na_to_a"
df.loc[(df["aro_initial_aa"] == "non_aromatic") & (df["aro_final_aa"] == "non_aromatic"), "d_aro"] = "na_to_na"

# Check if everything was ok
df["d_aro"].unique()

array(['na_to_na', 'na_to_a', 'a_to_na', 'a_to_a'], dtype=object)

<br>

<u> **Descriptores estructurales y evolutivos** </u>

Por último, se añaden los descriptores estructurales y evolutivos:
* `df["r_conserv"]`: que recogería la conservación evolutiva del residuo donde ocurre la mutación.
* `df["secondary_str"]`: recogería la estructura secundaria del canal Kv7.2 donde ocurre la mutación, es decir, si cae en una hélice alfa, coiled coil...etc. 

Para añadir la **(A) conservación evolutiva** se va a abrir el fichero donde se calcularon las conservaciones de cada posición para el caso de O43526.

In [40]:
# Load conservation file and define a df
condf = pd.read_csv('FINAL_plot_conservation.csv', header = "infer")

In [41]:
condf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 872 entries, 0 to 871
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Residue       872 non-null    int64  
 1   Conservation  872 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 13.8 KB


El archivo presenta dos columnas, una con la posición del residuo (`condf["Residue"]`) y otra con el valor de conservación calculado para ese mismo residuo (`condf["Conservation"]`):

In [42]:
condf.head()

,Residue,Conservation
0,1,0.61970
1,2,0.65838
2,3,0.71927
3,4,0.74279
4,5,0.73004


Se realiza un procesado del fichero según las necesidades:

In [43]:
# Create a dict with res as keys and conserv as valeus
d = {}
for v in condf.iterrows():
    res = int(v[1][0]) # residues
    con = float(v[1][1]) # conservation
    d[res] = con
    
# lists
res = [i for i in d.keys()]
con = [i for i in d.values()]
lpos = [i for i in df["position_aa"]]

Pare diseñar el descriptor, nos tenemos que quedar con los valores de conservación para los cuales exista un número de residuo (`consdf["Residue"]`) y una posición del residuo (`df["position_aa"]`) compartidas. Ej.: me quedaré con el valor de conservación correspondiendte cuando `consdf["Residue"]` = `df["position_aa"]`. Para eso:

In [44]:
lcons = []

for i in range(len(lpos)):
    for j in range(len(res)):
        if lpos[i] == res[j]:
            lcons.append(con[j])

# Create that column in df:
df["r_conserv"] = lcons 
df.head(10)

,PrimaryDisease,Mutationppt,initial_aa,final_aa,position_aa,topological_domain,functional_domain,char_initial_aa,char_final_aa,pol_initial_aa,...,hf_initial_aa,hf_final_aa,d_size,d_hf,d_vol,d_msa,d_charge,d_pol,d_aro,r_conserv
0,pathogenic_variant,M1T,M,T,1,Cytoplasmic,unknown_function,neutral,neutral,non_polar,...,1.9,-0.7,30.09,2.6,28.8,-31.4,neu_to_neu,np_to_p,na_to_na,0.61970
1,pathogenic_variant,M1V,M,V,1,Cytoplasmic,unknown_function,neutral,neutral,non_polar,...,1.9,4.2,32.06,-2.3,15.1,-15.7,neu_to_neu,np_to_np,na_to_na,0.61970
2,pathogenic_variant,L107F,L,F,107,S1,voltage_domain,neutral,neutral,non_polar,...,3.8,2.8,-34.02,1.0,-13.6,4.6,neu_to_neu,np_to_np,na_to_a,0.78350
3,pathogenic_variant,T114A,T,A,114,Extracelullar,voltage_domain,neutral,neutral,polar,...,-0.7,1.8,30.03,-2.5,18.1,-2.1,neu_to_neu,p_to_np,na_to_na,0.80304
4,pathogenic_variant,T114I,T,I,114,Extracelullar,voltage_domain,neutral,neutral,polar,...,-0.7,4.5,-12.05,-5.2,-29.6,27.8,neu_to_neu,p_to_np,na_to_na,0.80304
5,pathogenic_variant,E119G,E,G,119,Extracelullar,voltage_domain,negative_acidic,neutral,polar,...,-3.5,-0.4,72.06,-3.1,47.2,-5.4,neg_to_neu,p_to_p,na_to_na,0.76221
6,pathogenic_variant,S122L,S,L,122,Extracelullar,voltage_domain,neutral,neutral,polar,...,-0.8,3.8,-26.08,-4.6,-45.5,30.4,neu_to_neu,p_to_np,na_to_na,0.74631
7,pathogenic_variant,E130K,E,K,130,S2,voltage_domain,negative_acidic,positive_basic,polar,...,-3.5,-3.9,0.94,0.4,-20.8,-5.7,neg_to_pos,p_to_p,na_to_na,0.80182
8,pathogenic_variant,R144Q,R,Q,144,Cytoplasmic,voltage_domain,positive_basic,neutral,polar,...,-4.5,-3.5,28.05,-1.0,19.5,6.5,pos_to_neu,p_to_p,na_to_na,0.84035
9,pathogenic_variant,Y154D,Y,D,154,Cytoplasmic,voltage_domain,neutral,negative_acidic,non_polar,...,-1.3,-3.5,48.09,2.2,47.2,-14.9,neu_to_neg,np_to_p,a_to_na,0.88678


<br>

Por último, se incluye el descriptor de **(B) estructura secundaria** donde ocurre la mutación. Para localizar en qué estructura secundaria ocurre la mutación voy a hacer uso de herramientas bioinformáticas que me van a permitir predecir la estructura secundaria de la secuencia fasta de O43526. Esto lo tengo que hacer de esta manera puesto que no se encuentra completamente resuelta la estructura del canal a día de hoy. Con el fin de obtener la estructura secundaria se empleó el método predictor de estructura secundaria PROTEUS2 (http://www.proteus2.ca/proteus2/). Como opciones adicionales se especifica que se trata de una secuencia eucariota y le proporciono el PDB 7cr3.pdb para que haga *homollogy modelling*. Dejo por defecto la opción de *run energy minimization* porque puede mejorar la precisión del modelo aunque pueda tardar un poco más.

In [45]:
# output of proteus2
prot2 = "CCCCCCCCCCCCCCHHHHHHHHCCECCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHCCCCHHHTTTTTTTTTTTTTTTTTTTTTTCHHHHHHHHHCTTTTTTTTTTTTTTTTTTTTTTCCCCCCCCCCCCCHHHHHHHCTTTTTTTTTTTTTTTTTTCHHHHHHHHHHHHCTTTTTTTTTTTTTTTTCCHHHHHHHHHHHHCCCTTTTTTTTTTTTTTTTTTTTTTTTTCCCCCCCCTTTTTTTTTTTTTCCCCCCCCCCCTTTTTTTTTTTTTTTTTTTCTTTTTTTTCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCCCHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCCHHHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCECCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCECHHHHHHHHHHHHHHHHHCCCCCHHHHHHHHHHCHHHHHHHHHHHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCCCCCECCCCCCCCCCCCCCCCCCCCCCCCCCCEEEEEEEEEECCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCEEECCCCHHHHHHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCECCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC"
prot2

'CCCCCCCCCCCCCCHHHHHHHHCCECCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHCCCCHHHTTTTTTTTTTTTTTTTTTTTTTCHHHHHHHHHCTTTTTTTTTTTTTTTTTTTTTTCCCCCCCCCCCCCHHHHHHHCTTTTTTTTTTTTTTTTTTCHHHHHHHHHHHHCTTTTTTTTTTTTTTTTCCHHHHHHHHHHHHCCCTTTTTTTTTTTTTTTTTTTTTTTTTCCCCCCCCTTTTTTTTTTTTTCCCCCCCCCCCTTTTTTTTTTTTTTTTTTTCTTTTTTTTCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCCCHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCCHHHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCECCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCECHHHHHHHHHHHHHHHHHCCCCCHHHHHHHHHHCHHHHHHHHHHHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCCCCCECCCCCCCCCCCCCCCCCCCCCCCCCCCEEEEEEEEEECCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCEEECCCCHHHHHHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCECCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC'

Se realiza el cambio mencionado con anterioridad. Pare diseñar el descriptor, nos tenemos que quedar con los valores de conservación para los cuales exista un número de residuo (`consdf["Residue"]`) y una posición del residuo (`df["position_aa"]`) compartidas. Ej.: me quedaré con el valor de conservación correspondiendte cuando `consdf["Residue"]` = `df["position_aa"]`. Para eso:

In [46]:
lpos = [i for i in df["position_aa"]] # mutation positions in db
lss = []

for i in range(len(prot2)):
    for j in lpos:
        if i+1 == j:
            lss.append(prot2[i+1])

# Create that column in df:
df["sec_str"] = lss
df["sec_str"].unique()

array(['C', 'H', 'T', 'E'], dtype=object)

Por último, se mappean los valores únicos para que no se confundan con aminoácidos. Se mappea según la nomenclatura de PROTEUS2:
* H = Helix.
* E = Beta Strand.
* C = Coil.
* T = Memebrane helix.

In [47]:
ss_map = {
    'C':'coil',
    'T':'membrane_helix',
    'H':'helix',
    'E':'beta_strand'  
}

# pol_initial_aa
df["sec_str"].replace(ss_map, inplace=True)
print(df["sec_str"].unique())

['coil' 'helix' 'membrane_helix' 'beta_strand']


El DataFrame final sería el siguiente:

In [48]:
# Show first records
df.head(10)

,PrimaryDisease,Mutationppt,initial_aa,final_aa,position_aa,topological_domain,functional_domain,char_initial_aa,char_final_aa,pol_initial_aa,...,hf_final_aa,d_size,d_hf,d_vol,d_msa,d_charge,d_pol,d_aro,r_conserv,sec_str
0,pathogenic_variant,M1T,M,T,1,Cytoplasmic,unknown_function,neutral,neutral,non_polar,...,-0.7,30.09,2.6,28.8,-31.4,neu_to_neu,np_to_p,na_to_na,0.61970,coil
1,pathogenic_variant,M1V,M,V,1,Cytoplasmic,unknown_function,neutral,neutral,non_polar,...,4.2,32.06,-2.3,15.1,-15.7,neu_to_neu,np_to_np,na_to_na,0.61970,coil
2,pathogenic_variant,L107F,L,F,107,S1,voltage_domain,neutral,neutral,non_polar,...,2.8,-34.02,1.0,-13.6,4.6,neu_to_neu,np_to_np,na_to_a,0.78350,coil
3,pathogenic_variant,T114A,T,A,114,Extracelullar,voltage_domain,neutral,neutral,polar,...,1.8,30.03,-2.5,18.1,-2.1,neu_to_neu,p_to_np,na_to_na,0.80304,coil
4,pathogenic_variant,T114I,T,I,114,Extracelullar,voltage_domain,neutral,neutral,polar,...,4.5,-12.05,-5.2,-29.6,27.8,neu_to_neu,p_to_np,na_to_na,0.80304,coil
5,pathogenic_variant,E119G,E,G,119,Extracelullar,voltage_domain,negative_acidic,neutral,polar,...,-0.4,72.06,-3.1,47.2,-5.4,neg_to_neu,p_to_p,na_to_na,0.76221,helix
6,pathogenic_variant,S122L,S,L,122,Extracelullar,voltage_domain,neutral,neutral,polar,...,3.8,-26.08,-4.6,-45.5,30.4,neu_to_neu,p_to_np,na_to_na,0.74631,helix
7,pathogenic_variant,E130K,E,K,130,S2,voltage_domain,negative_acidic,positive_basic,polar,...,-3.9,0.94,0.4,-20.8,-5.7,neg_to_pos,p_to_p,na_to_na,0.80182,coil
8,pathogenic_variant,R144Q,R,Q,144,Cytoplasmic,voltage_domain,positive_basic,neutral,polar,...,-3.5,28.05,-1.0,19.5,6.5,pos_to_neu,p_to_p,na_to_na,0.84035,membrane_helix
9,pathogenic_variant,Y154D,Y,D,154,Cytoplasmic,voltage_domain,neutral,negative_acidic,non_polar,...,-3.5,48.09,2.2,47.2,-14.9,neu_to_neg,np_to_p,a_to_na,0.88678,membrane_helix


<br>
<br>
<br> 

### 2. Eliminación de duplicados, unificación del formato y corrección de errores <a id="1.2"></a>

En este apartado cabe destacar que no es necesario unificar el formato y corregir errores porque el fichero del que se parte ha sido revisado anteriormente. Sin embargo, sí se van a comprobar posibles duplicados:

In [49]:
# Check for duplicates 
df[df["Mutationppt"].duplicated()]

# Remove duplicates if needed
#ch_df = ch_df.drop_duplicates()

,PrimaryDisease,Mutationppt,initial_aa,final_aa,position_aa,topological_domain,functional_domain,char_initial_aa,char_final_aa,pol_initial_aa,...,hf_final_aa,d_size,d_hf,d_vol,d_msa,d_charge,d_pol,d_aro,r_conserv,sec_str


La base de datos NO presenta valores duplicados.

<br>
<br>
<br> 

### 3. Preprocesado final para los modelos de ML <a id="1.3"></a>

Por último, se realiza un preprocesado final para mantener los descriptores que se han considerado relevantes para el diseño de algoritmos de *Machine Learning*. Ej.: interesa solo quedarse con la columna de cambio de carga `df["d_charge"]`, por lo que se deberían de eliminar las columnas de las que se obtuvo: `df["mw_initial_aa"]` y `df["mw_final_aa"]`.

Así, se adapta la tabla eliminando una serie de columnas y cambiando el nombre de las restantes para una mejor interpretación. Se crea una función que cumpla este objetivo:

In [50]:
def preprocessing_df(df):
    """
    Description: 
        Takes as input the database with the designed columns. 
        It eliminates those columns that are useless, renames the
        remaining ones and performs a binary coding of the label.
    
    Input: 
        Original dataframe
        
    Return:
        Modified dataframe
    """
    
    # Change column names
    df.columns  = ['type_of_variant', 'mutation', 'initial_aa', 'final_aa', 'position_aa', 'topological_domain',
       'functional_domain', 'char_initial_aa', 'char_final_aa', 'pol_initial_aa', 'pol_final_aa',
       'aro_initial_aa', 'aro_final_aa', 'mw_initial_aa', 'mw_final_aa',
       'v_e_initial_aa', 'v_e_final_aa', 'pol_e_initial_aa', 'pol_e_final_aa',
       'ip_e_initial_aa', 'ip_e_final_aa', 'hf_e_initial_aa', 'hf_e_final_aa',
       'msa_e_initial_aa', 'msa_e_final_aa', 'hf_initial_aa', 'hf_final_aa',
       'd_size', 'd_hf', 'd_vol', 'd_msa','d_charge', 'd_pol', 'd_aro', 'residue_conserv', 'secondary_str']
    
    # Create binary label
    l =  []

    for i in df["type_of_variant"]:
        if i == "pathogenic_variant":
            l.append(int(1))

        if i == "benign_variant":
            l.append(int(0))
        
    #df = df.drop(["type_of_variant"], axis = 1) # Delete original column
    df["label"] = l
    
    # Delete extra columns 
    df = df.drop(['position_aa','char_initial_aa', 'char_final_aa', 'pol_initial_aa', 'pol_final_aa',
       'aro_initial_aa', 'aro_final_aa', 'mw_initial_aa', 'mw_final_aa',
       'v_e_initial_aa', 'v_e_final_aa', 'pol_e_initial_aa', 'pol_e_final_aa',
       'ip_e_initial_aa', 'ip_e_final_aa', 'hf_e_initial_aa', 'hf_e_final_aa',
       'msa_e_initial_aa', 'msa_e_final_aa', 'hf_initial_aa', 'hf_final_aa'], axis = 1)
    
    return df

In [51]:
wdf = preprocessing_df(df)

Para convertir el fichero en un archivo CSV, es necesario ejecutar:

In [52]:
## - To CSV
#wdf.to_csv('FINAL_DDBB_v3.csv', sep=',', index=False)